In [1]:
import numpy as np
from scipy.linalg import eig
import copy
import math

#### Question 1
Prove the Epsilon-Greedy Policy Improvement Theorem (we sketched the proof in Class)

$${q_\pi(s, \pi'(s)) = \Sigma_{a \in A}\pi'(a|s)q_{\pi}(s,a)}$$
$${q_\pi(s, \pi'(s)) = \epsilon/m\Sigma_{a \in A}q_{\pi}(s,a) + (1 - \epsilon)max_{a \in A}q_{\pi}(s,a) \\  
\geq
\frac{\epsilon}{m}\Sigma_{a \in A}q_{\pi}(s,a) + (1 - \epsilon)\Sigma_{a \in A}\frac{\pi(a|s) - \epsilon/m}{1-\epsilon}q_\pi(a|s) \\
= \Sigma_{a \in A}\pi(a|s)q_{\pi}(s,a) = v_\pi(s)}$$

#### Question 2
Provide (with clear mathematical notation) the defintion of GLIE (Greedy in the Limit with Infinite Exploration)

Definition for GLIE (Greedy in the limit with infinite exploration):
All state-action pairs are explored infinitely many times, ${lim_{k -> \infty}N_k(s,a) = \infty}$, and the policy converges on a greedy policy. ${lim_{k -> \infty}\pi_k(a|s) = 1(a = argmax_{a' \in A}Q_k(s, a'))}$

GLIE Monte-Carlo Control: Sample kth episode using ${\pi}$, for each state ${S_t}$ and action ${A_t}$ in the episode,  
$${N(S_t, A_t) \leftarrow N(S_t, A_t) + 1}$$
$${Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \frac{1}{N(S_t, A_t)}(G_t - Q(S_t, A_t))}$$  
Improve policy based on new action-value function:
$${\epsilon \leftarrow \frac{1}{k}}$$
$${\pi \leftarrow \epsilon-greedy(Q)}$$

#### Question 3
Implement the tabular SARSA and tabular SARSA(Lambda) algorithms  

In [17]:
from scipy.stats import rv_discrete
from typing import Mapping, Set, Tuple, Sequence, Any, Callable, TypeVar
X = TypeVar('X')
A = TypeVar('A')
S = TypeVar('S')

def random_generator_based_on_prob(prob_dict: Mapping[S, float]) -> Callable[[], S]:
    """
    A generator of states/ state-action pairs based on predefined probabilities
    It serves as a simulator to real world
    @return an instance with corresponding probabilities
    """
#     rv_discrete is a base class to construct specific distribution classes and instances 
#     for discrete random variables. It can also be used to construct an arbitrary distribution
#     defined by a list of support points and corresponding probabilities.
    outcomes, probabilities = zip(*prob_dict.items())
    rvd = rv_discrete(values=(range(len(outcomes)), probabilities))

    return lambda rvd=rvd, outcomes=outcomes: outcomes[rvd.rvs(size=1)[0]]

In [18]:
simulator = random_generator_based_on_prob({'A': 0.2, 'B':0.7, 'C':0.1}) # a test case to simulate
# in reality, the key should be state and the value is the correspoding probability of arriving in that state give teh state and action taken

In [24]:
class MDP:
    """
    The MDP is defined to initalize with a state_action_dict and state_reward_dict
    The MDP is conpatible with the requirement for Monte Carlo or TD learning algorithm
    """
    def __init__(self,state_action_dict: Mapping[S, Set[A]], terminal_states: Set[S], state_reward_gen_dict, gamma: float) -> None:
        
        # a mapping from a tuple of (state, action) to the reward
        self.state_action_dict: Mapping[S, Set[A]] = state_action_dict
        
        self.terminal_states: Set[S] = terminal_states
        self.state_reward_gen_dict: Type1 = state_reward_gen_dict
        self.state_action_func = lambda x: self.state_action_dict[x],
        self.gamma=gamma,
        self.terminal_state_func = lambda x: x in self.terminal_states,
        self.state_reward_gen_func = lambda x, y: self.state_reward_gen_dict[x][y](),
        
        # initialize the state generator with equal probability of each state 
        self.init_state_generator = random_generator_based_pn_prob(
                {s: 1. / len(self.state_action_dict) for s
                 in self.state_action_dict.keys()}
            ),
        
        # initialize the (state, action) generator with equal probability of each pair
        self.init_state_action_generator = random_generator_based_on_prob(
                {(s, a): 1. / sum(len(v) for v
                                  in self.state_action_dict.values())
                 for s, v1 in self.state_action_dict.items() for a in v1}
        )


In [27]:
# some class prototype defined, e.g MDP and Policy
# the function definition of Policy is in accordance with https://github.com/coverdrive/MDP-DP-RL/
# with a simplified test harness
from operator import itemgetter
from typing import Mapping, Set, Tuple, Sequence, Any, Callable, TypeVar, Dict

def epsilon_greedy(action_value_dict, epsilon: float) -> Mapping[A, float]:
    """
    Using Epsilon-Greedy method to select the action based on the value
    @return: the function returns the a dictionary of a action and its probability to be selected
    """
    max_act = max(action_value_dict.items(), key=itemgetter(1))[0]
    m = len(action_value_dict)
    if epsilon == 0:
        return {max_act: 1.}
    else:
        # with 1 / epsilon to select a random  
        return {action: epsilon / m + (1. - epsilon if a == max_act else 0.) for action in action_value_dict.keys()}




class Policy:

    def __init__(self, data: Dict[S, Mapping[A, float]]) -> None:
        self.policy_data = data

    def get_state_probabilities(self, state: S) -> Mapping[A, float]:
        return self.policy_data[state]
    
    def update_state_action_to_greedy(self, state, action_value_dict):
        self.policy_data[state] = {max(action_value_dict.item(), key=itemgetter(1))[0]:1.0}

    def get_state_action_probability(self, state: S, action: A) -> float:
        return self.get_state_probabilities(state).get(action, 0.)

    def edit_state_action_to_epsilon_greedy(
        self,
        state: S,
        action_value_dict: Mapping[A, float],
        epsilon: float
    ) -> None:
        self.policy_data[state] = self.epsilon_greedy(action_value_dict, epsilon)
    
    def __repr__(self):
        return self.policy_data.__repr__()

    def __str__(self):
        return self.policy_data.__str__()



In [29]:
class SARSA:

    def __init__(self, mdp: MDP, epsilon: float, learning_rate: float, learning_rate_decay: float, \
                 lambd: float, num_episodes: int, max_steps: int, initial_policy: Policy):

        self.mdp = mdp,
        self.lambd = lambd
        self.epsilon = epsilon,
        self.num_episodes = num_episodes,
        self.max_steps = max_steps
        self.learning_rate: float = learning_rate
        self.gamma_lambda = self.mdp.gamma * lambd
        
        self.policy = initial_policy
    
    def epsilon_greedy(self, policy, current_state, state_value_dict, epsilon):
        # the probability of selecting certain action based on the policy
        actions = policy.get_state_probabilities(current_state)
        
        # the action chosen based on greedy algorithm
        max_act = current_state
        for action in actions:
            if state_value_dict[current_state][action] > state_value_dict[current_state][max_act]:
                max_act = action
        
        m = len(action_value_dict)
        policy.edit_state_action_to_epsilon_greedy(current_state, self.mdp.action_value_dict)
        if epsilon == 0:
            return {max_act: 1.}
        else:
            # with 1 / epsilon to select a random  
            return {action: epsilon / m + (1. - epsilon if a == max_act else 0.) for action in action_value_dict.keys()}
        
        
    def update_one_episode(self, current_state, current_action, state_action_value_dict, policy):
        # before updating using SARSA
        curt_state = current_state
        online_state_value_dict = copy.deepcopy(state_action_value_dict)
        curt_action = current_action
        for step in range(self.max_steps):
            # generate next step based on epsilon-greedy
            actions = self.epsilon_greedy(policy, current_state, online_state_value_dict, self.epsilon)
            # here the next action to take is based on the probability generated by the epsilon greedy
            next_action_generator = random_generator_based_on_prob(actions)
            action_to_take = next_action_generator()
            
            next_state, reward = self.mdp.state_reward_gen_dict[current_state][curt_action]()
            
            # online update for Q(S,A)
            online_state_value_dict[curt_state][curt_action] = online_state_value_dict[curt_state][curt_action] + \
            self.learning_rate * (reward + self.gamma * online_state_value_dict[next_state][action_to_take] - online_state_value_dict[curt_state][curt_action])
            if curt_state in self.mdp.terminal_states:
                break
            else:
                curt_state = next_state
                curt_action = action_to_take
        # update the policy and the Q(S,A) accordingly
        return (policy, online_state_value_dict)
    
    def SARSA_update(self, pol:Policy, inital_state, intial_action):
        """
        Update the policy and Q(S,A) using SARSA online update
        """
        
        state_action_value_dict = self.mdp.state_action_dict
        

        for episode in range(self.max_episodes):
            current_state = initial_state
            current_action = initial_action
            (policy, state_action_value_dict) = self.update_one_episode(current_state, \
                                                                        current_action, \
                                                                        copy.deepcopy(state_action_value_dict), \
                                                                        copy.deepcopy(policy))
            

        return (policy, state_action_value_dict)
    
    
    # a helper function used to calculate the G_t_lambd value for current state
    def calculate_g_t_lambd(step, g_t_list, lambd):
        total_step = len(g_t_list)
        g_t_lambd = 0
        current_coeff = 1
        for i in range(step, total_step, 1):
            g_t_lambd += current_coeff * g_t_list[i]
            current_coeff *= lamba
        return (1.0 - lambd) * g_t_lambd
    
    def get_one_TDLambda_update(self, pol: Policy, state_value_dict, action_generator_for_each_state):
        # Update MDP by running one episode of TD-lambda learning
        
        
        # preserve the state value/ action information of the current MDP
        state_action_dict = self.mdp.state_action_dict
        state_value = copy.deepcopy(state_value_dict)
        
        # generate the trace of this episode
        state_list = []
        action_list = []
        value_list = []
        
        # use the initial state generate by MDP, it could also be initialized to a fixed state
        current_state = self.mdp.init_state_generator()
        
        for step in range(self.max_steps):
            action_to_take = action_generator_for_each_state[current_state]()
            next_state, reward = self.mdp.state_reward_gen_dict[current_state][action_to_take]()
            state_list.append(current_state)
            action_list.append(action_to_take)
            value_list.append(reward)
            if current_state in self.mdp.terminal_states:
                break
            else:
                # increment by one step
                current_state = next_state
        
        g_t_list = []
        # calculating G_t in a backward manner
        for i in range(len(state_list) - 1, -1. -1):
            if i == len(state_list) - 1:
                current_g_t = value_list[i]
            else:
                current_g_t = value_list[i] + g_t_list[-1] * self.lambd
            g_t_list.append(current_g_t)
        # reverse the g_t_list to match the time
        g_t_list = g_t_list[::-1]
        
        #  update the value function 
        for step in range(len(state_list)):
            # this time only consider the state visited in this episode
            g_t_lambd = calculate_g_t_lambd(step, g_t_list, lambd)
            state_value[state_list[step]] += self.learning_rate * (g_t_lambd - state_value[state_list[step]])
        
        return state_value
    
    # the update function running over all the episodes
    def update(self, pol:Policy):
        state_action_dict = self.mdp.state_action_dict
        # initate the value for each state as 0
        V_s = {s: 0. for s in sa_dict.keys()}
        
        # action generator for each state
        action_generator_for_each_state = {s: random_generator_based_on_prob(pol.get_state_probabilities(s))
                        for s in sa_dict.keys()}
        episodes = 0
        updates = 0

        for episode in range(self.max_episodes):
            et_dict = {s: 0. for s in sa_dict.keys()}
            state = self.mdp_rep.init_state_gen()
            steps = 0
            terminate = False

            V_s = self.get_one_TDLambda_update(pol, copy.deepcopy(V_s), action_generator_for_each_state)

        return V_s
    

#### Question 4
Implement the tabular Q-Learning algorithm

For Q-learning, no importance sampling is required  
Next action is chosen using behaviour policy ${A_{t+1} ~ \mu(\circ|S_t)}$

In [ ]:
class Q_leaning:

    def __init__(self, mdp: MDP, epsilon: float, learning_rate: float, learning_rate_decay: float, \
                 lambd: float, num_episodes: int, max_steps: int, initial_policy: Policy):

        self.mdp = mdp,
        self.lambd = lambd
        self.epsilon = epsilon,
        self.num_episodes = num_episodes,
        self.max_steps = max_steps
        self.learning_rate: float = learning_rate
        self.gamma_lambda = self.mdp.gamma * lambd
        
        self.policy = initial_policy
    
    def epsilon_greedy(self, policy, current_state, state_value_dict, epsilon):
        # the probability of selecting certain action based on the policy
        actions = policy.get_state_probabilities(current_state)
        
        # the action chosen based on greedy algorithm
        max_act = current_state
        for action in actions:
            if state_value_dict[current_state][action] > state_value_dict[current_state][max_act]:
                max_act = action
        
        m = len(action_value_dict)
        policy.edit_state_action_to_epsilon_greedy(current_state, self.mdp.action_value_dict)
        if epsilon == 0:
            return {max_act: 1.}
        else:
            # with 1 / epsilon to select a random  
            return {action: epsilon / m + (1. - epsilon if a == max_act else 0.) for action in action_value_dict.keys()}
    
    # compared with SARSA, Q-learning has a separate method to choose next_action based on greedy policy
    def Q_learning_greedy(self, policy, current_state, state_value_dict):
        # the probability of selecting certain action based on the policy
        actions = policy.get_state_probabilities(current_state)
        
        # the action chosen based on greedy algorithm
        max_act = current_state
        for action in actions:
            if state_value_dict[current_state][action] > state_value_dict[current_state][max_act]:
                max_act = action
        
        policy = policy.update_state_action_to_greedy(current_state, self.mdp.action_value_dict)
        return {max_act: 1.}

        
    def update_one_episode(self, current_state, current_action, state_action_value_dict, policy):
        # Use Q leaning to update the state_action_dictiary and change the policy
        curt_state = current_state
        online_state_value_dict = copy.deepcopy(state_action_value_dict)
        curt_action = current_action
        for step in range(self.max_steps):
            # generate next step based on epsilon-greedy
            actions = self.Q_learning_greedy(policy, current_state, online_state_value_dict)
            # here the next action to take is based on the probability generated by the epsilon greedy
            next_action_generator = random_generator_based_on_prob(actions)
            action_to_take = next_action_generator()
            
            next_state, reward = self.mdp.state_reward_gen_dict[current_state][curt_action]()
            
            # online update for Q(S,A)
            online_state_value_dict[curt_state][curt_action] = online_state_value_dict[curt_state][curt_action] + \
            self.learning_rate * (reward + self.gamma * online_state_value_dict[next_state][action_to_take] - online_state_value_dict[curt_state][curt_action])
            if curt_state in self.mdp.terminal_states:
                break
            else:
                curt_state = next_state
                curt_action = action_to_take
        # update the policy and the Q(S,A) accordingly
        return (policy, online_state_value_dict)
    
    def Q_leaning_update(self, pol:Policy, inital_state, intial_action):
        """
        Update the policy and Q(S,A) using SARSA online update
        """
        
        state_action_value_dict = self.mdp.state_action_dict
        

        for episode in range(self.max_episodes):
            current_state = initial_state
            current_action = initial_action
            (policy, state_action_value_dict) = self.update_one_episode(current_state, \
                                                                        current_action, \
                                                                        copy.deepcopy(state_action_value_dict), \
                                                                        copy.deepcopy(policy))
            

        return (policy, state_action_value_dict)
    
    
    # a helper function used to calculate the G_t_lambd value for current state
    def calculate_g_t_lambd(step, g_t_list, lambd):
        total_step = len(g_t_list)
        g_t_lambd = 0
        current_coeff = 1
        for i in range(step, total_step, 1):
            g_t_lambd += current_coeff * g_t_list[i]
            current_coeff *= lamba
        return (1.0 - lambd) * g_t_lambd
    
    def get_one_TDLambda_update(self, pol: Policy, state_value_dict, action_generator_for_each_state):
        # Update MDP by running one episode of TD-lambda learning
        
        
        # preserve the state value/ action information of the current MDP
        state_action_dict = self.mdp.state_action_dict
        state_value = copy.deepcopy(state_value_dict)
        
        # generate the trace of this episode
        state_list = []
        action_list = []
        value_list = []
        
        # use the initial state generate by MDP, it could also be initialized to a fixed state
        current_state = self.mdp.init_state_generator()
        
        for step in range(self.max_steps):
            action_to_take = action_generator_for_each_state[current_state]()
            next_state, reward = self.mdp.state_reward_gen_dict[current_state][action_to_take]()
            state_list.append(current_state)
            action_list.append(action_to_take)
            value_list.append(reward)
            if current_state in self.mdp.terminal_states:
                break
            else:
                # increment by one step
                current_state = next_state
        
        g_t_list = []
        # calculating G_t in a backward manner
        for i in range(len(state_list) - 1, -1. -1):
            if i == len(state_list) - 1:
                current_g_t = value_list[i]
            else:
                current_g_t = value_list[i] + g_t_list[-1] * self.lambd
            g_t_list.append(current_g_t)
        # reverse the g_t_list to match the time
        g_t_list = g_t_list[::-1]
        
        #  update the value function 
        for step in range(len(state_list)):
            # this time only consider the state visited in this episode
            g_t_lambd = calculate_g_t_lambd(step, g_t_list, lambd)
            state_value[state_list[step]] += self.learning_rate * (g_t_lambd - state_value[state_list[step]])
        
        return state_value
    
    # the update function running over all the episodes
    def update(self, pol:Policy):
        state_action_dict = self.mdp.state_action_dict
        # initate the value for each state as 0
        V_s = {s: 0. for s in sa_dict.keys()}
        
        # action generator for each state
        action_generator_for_each_state = {s: random_generator_based_on_prob(pol.get_state_probabilities(s))
                        for s in sa_dict.keys()}
        episodes = 0
        updates = 0

        for episode in range(self.max_episodes):
            et_dict = {s: 0. for s in sa_dict.keys()}
            state = self.mdp_rep.init_state_gen()
            steps = 0
            terminate = False

            V_s = self.get_one_TDLambda_update(pol, copy.deepcopy(V_s), action_generator_for_each_state)

        return V_s
    